# Imports

In [ ]:
import iris_utilities as iris

# Main

In [ ]:

imagepath = "Dataset/VISA_Iris/VISA_Iris/S0001_F_30/R/1.bmp"

# hough
image1, radius, success = iris.processing(imagepath, 60)

# remove reflection
imageN = iris.remove_reflection(image1)

# daugman
sheet = iris.generate_rubber_sheet_model(image1)
sheetN = iris.generate_rubber_sheet_model(imageN)

# dwt reflection
dwt_feature = iris.feature_extraction(sheet)

# dwt no reflection
dwt_featureN = iris.feature_extraction(sheetN)

print("DWT Features:", dwt_feature)
print("DWT Features N:", dwt_featureN)

fig, axes = iris.plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(imageN)
axes[0].set_title('Iris Image')
axes[0].axis('off')  # Hide axes for cleaner presentation

axes[1].imshow(sheetN)
axes[1].set_title('Daugman Rubber Sheet')
axes[1].axis('off')

iris.plt.tight_layout()
iris.plt.show()

# TEST

imagepath = "Dataset/VISA_Iris/VISA_Iris/S0001_F_30/L/2.bmp"

eye_image = iris.cv2.imread(imagepath, iris.cv2.IMREAD_GRAYSCALE)

eye_circles = iris.cv2.HoughCircles(
    eye_image, iris.cv2.HOUGH_GRADIENT, 2, 100,  minRadius=90, maxRadius=200)

iris_coordinates = None

if eye_circles is not None:
    circle = eye_circles[0][0]
    iris_coordinates = (circle[0], circle[1])

if iris_coordinates is not None:
    x = int(iris_coordinates[0])
    y = int(iris_coordinates[1])

    w = int(round(circle[2]) + 10)
    h = int(round(circle[2]) + 10)

    image_hough_processed = eye_image[y-h:y+h, x-w:x+w]
    iris_image_to_show = iris.cv2.resize(
        image_hough_processed, (image_hough_processed.shape[1]*2, image_hough_processed.shape[0]*2))

q = iris.np.arange(0.00, iris.np.pi*2, 0.01)  # theta
inn = iris.np.arange(0, int(iris_image_to_show.shape[0]/2), 1)  # radius

cartisian_image = iris.np.empty(
    shape=[inn.size, int(iris_image_to_show.shape[1]), 3])
m = iris.interp1d([iris.np.pi*2, 0], [0, iris_image_to_show.shape[1]])

for r in inn:
    for t in q:
        polarX = int((r * iris.np.cos(t)) + iris_image_to_show.shape[1]/2)
        polarY = int((r * iris.np.sin(t)) + iris_image_to_show.shape[0]/2)
        cartisian_image[r][int(m(t) - 1)] = iris_image_to_show[polarY][polarX]

cartisian_image = cartisian_image.astype('uint8')

fig, axes = iris.plt.subplots(1, 3, figsize=(12, 5))
axes[0].imshow(eye_image, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(iris_image_to_show, cmap='gray')
axes[1].set_title('Haar Iris Cascade Classifier Detection')
axes[1].axis('off')

axes[2].imshow(cartisian_image, cmap='gray')
axes[2].set_title('Cropped Eye')
axes[2].axis('off')

iris.plt.tight_layout()
iris.plt.show()

# TEST
eye_images = iris.parse_iris_dataset()
iris.iris_detection(eye_images, display=True)

# Parse the iris dataset to get the list of eye images
eye_images = iris.parse_iris_dataset()

# Call the iris_detection function to detect irises in the eye images and display the results
iris.iris_detection(eye_images, display=1)